# 필요한 라이브러리 import

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import re
import numpy as np
import datetime

# 필요한 df 준비

In [4]:
df1=pd.read_csv(r'C:\Users\Playdata\Desktop\programers_cp949_1.csv', encoding='cp949')
df2=pd.read_csv(r"C:\Users\Playdata\Desktop\wanted_cp949_1.csv", encoding='cp949')
df3=pd.read_csv(r"C:\Users\Playdata\Desktop\jumpit_cp949_1.csv", encoding='cp949')

In [5]:
df=[df1,df2, df3]
df=pd.concat(df)
df=df[['공고명', '회사명', '직무', '마감일', '근무지', '기술스택', '링크']]

In [6]:
# 중복 공고가 들어가있는 링크 전체 리스트
link_set = set(df['링크'].unique().tolist())

In [7]:
now_df=pd.read_csv(r'C:\Users\Playdata\Desktop\final_true_address.csv', encoding='cp949')
now_df=now_df[['공고명', '회사명', '직무', '마감일', '근무지', '기술스택', '링크','평점','지역']]

In [8]:
# 새로운 공고 판별
new_df=pd.DataFrame(columns=df.columns)

In [9]:
#삭제되는 공고 판별
delete_df = pd.DataFrame(columns=['공고명', '회사명', '직무', '마감일', '근무지', '기술스택', '링크', '평점'])

In [10]:
# 현재 가지고 있는 공고
present_set = set()

# 함수

In [11]:
# 직무함수
def classify_position(position):
    if position in ['머신러닝', '인공지능', '머신러닝 엔지니어']:
        return 'AI/ML'
    elif position in ['웹 개발자','웹 풀스택']:
        return '웹개발'
    elif position in ['데이터 엔지니어','데이터 엔지니어링','빅데이터 엔지니어','BI 엔지니어']:
        return '데이터 엔지니어'
    elif position in ['데브옵스','DevOps/시스템 관리자']:
        return 'DevOps'
    elif position in ['서버/백엔드','PHP 개발자','루비온레일즈 개발자', '서버 개발자', '자바 개발자', 'C,C++ 개발자','파이썬 개발자','Node.js 개발자','DBA']:
        return '백엔드'
    elif position in ['프론트엔드', '프론트엔드 개발자']:
        return '프론트엔드'
    elif position in ['안드로이드', '안드로이드 개발자', 'iOS', 'iOS 개발자' ]:
        return '모바일'

In [12]:
# 기술함수
def classify_skill(skill):
    if skill in [ 'AWS','AWS Alexa','AWS Amplify','AWS CloudFront','AWS DynamoDB','AWS EC2','AWS ECS','AWS EKS','AWS Lambda',
                 'AWS RDS','AWS S3','AWS Serverless Application Repository','AWS Simple Notification Service(AWS SNS)',
                 'AWS Simple Queue Service(AWS SQS)','Amazon Web Service','Amazon Web Services(AWS)','AWS App Mesh',
                 'AWS Cloud Development Kit','AWS CodeCommit','AWS IoT Device Management','AWS Shell','Amazon API Gateway',
                 'Amazon CloudFront','Amazon Connect','Amazon DynamoDB','Amazon EC2','Amazon EKS','Amazon RDS','Amazon RDS for PostgreSQL',
                 'Amazon Route 53','Amazon S3','aws','amazon dynamodb','Amazon SQS', 'Yocto','aws batch', 'EC2' ,'EMR']:
        return 'AWS'
    elif skill in ['Kotlin','Android','Android Studio','Android OS','Android SDK', 'android os']:
        return 'Kotlin'
    elif skill in ['Angular','AngularJS','angular 2','Angular 2']:
        return 'Angular'
    elif skill in ['Apache','Apache HTTP Server','Apache Tomcat','Apache Traffic Server', 'apache tomcat']:
        return 'Apache'
    elif skill in ['Apache Hadoop','Hadoop']:
        return 'Hadoop'
    elif skill in ['Apache Kafka','Kafka','kafka']:
        return 'Kafka'
    elif skill in [ 'Apache Spark','Spark']:
        return 'Spark'
    elif skill in ['Apache ZooKeeper']:
        return 'ZooKeeper'
    elif skill in [ 'Azure','Azure Computer Vision','Azure Emotion','AZURE', 'Microsoft Azure']:
        return 'Azure'
    elif skill in ['C#']:
        return 'C#'
    elif skill in [ 'C', 'C / C++','C++','c','c++','Mfc','Mantis','Embedded C']:
        return 'C / C++'
    elif skill in ['AutoCAD','CAD']:
        return 'CAD'
    elif skill in [ 'CSS','CSS3','css 3','CSS 3','Tailwind CSS']:
        return 'CSS'
    elif skill in [ 'ElasticSearch','Elasticsearch', 'elasticsearch ','elasticsearch']:
        return 'ElasticSearch'
    elif skill in [ 'FPGA','FPGA 프로토 타이핑']:
        return 'FPGA'
    elif skill in ['Figma']:
        return 'Figma'
    elif skill in ['GCP','GCP(Google Cloud Platform)','GCPs','Google Cloud Platform']:
        return 'GCP'
    elif skill in ['GIS','GIS 시스템','GIS 응용 프로그램']:
        return 'GIS'
    elif skill in [ 'Git','GitHub','GitLab','Github','git','gitlab',
                  'Gerrit Code Review','GitHub Actions','GitLab CI']:
        return 'Git'
    elif skill in ['Gradle']:
        return 'Gradle'
    elif skill in ['GraphQL']:
        return 'GraphQL'
    elif skill in ['HAProxy']:
        return 'HAProxy'
    elif skill in ['HLSL']:
        return 'HLSL'
    elif skill in [ 'HTML','HTML5','html5']:
        return 'HTML'
    elif skill in ['HTTP']:
        return 'HTTP'
    elif skill in [ 'JSP','JSP 개발', 'jsp']:
        return 'JSP'
    elif skill in ['Java', 'java']:
        return 'Java'
    elif skill in [ 'Node.js', 'NodeJS','Mongoose', 'node.js']:
        return 'Node.js'
    elif skill in [ 'NumPy','Numpy']:
        return 'NumPy'
    elif skill in ['ORCAD']:
        return 'ORCAD'
    elif skill in ['Pandas']:
        return 'Pandas'
    elif skill in ['Python','python']:
        return 'Python'
    elif skill in ['React',  'React.js','ReactJS','react']:
        return 'React'
    elif skill in [ 'Redux','Redux-Saga','Redux-Thunk','Redux.js','React.js Boilerplate', 'redux-saga']:
        return 'Redux'
    elif skill in ['Ruby']:
        return 'Ruby'
    elif skill in ['Ruby on Rails']:
        return 'Ruby on Rails'
    elif skill in [ 'SAP','SAP ERP','SAP FI','SAP FICO','SAP HR','SAP MM','SAP SD','SAP 구현']:
        return 'SAP'
    elif skill in [ 'Shell','Shell Script','Shell Scripting']:
        return 'Shell'
    elif skill in [  'Hibernate','JPA','JPA(Java Persistent API)', 'Spring', 
                   'Spring Batch','Spring Boot','Spring Cloud','Spring Framework',
                  'spring','spring boot','spring framework', 'Spring MVC','Spring Data JPA']:
        return 'Spring'
    elif skill in [ 'Vue.JS','Vue.js','VueJS','vue.js']:
        return 'Vue.JS'
    elif skill in [ '.NET','.NET Core','.net']:
        return '.NET'
    elif skill in [ 'ASP', 'ASP .NET','ASP.NET']:
        return 'ASP.NET'
    elif skill in [ 'Linux', 'Linux kernel','CentOS', 'centos', 'linux','Embedded Linux']:
        return 'Linux'
    elif skill in ['UNIX','Unix']:
        return 'UNIX'
    elif skill in ['PyTorch','Pytorch', 'pytorch']:
        return 'PyTorch'
    elif skill in ['Flutter','flutter']:
        return 'Flutter'
    elif skill in ['Nest.js','NestJS']:
        return 'Nest.js'
    elif skill in ['Objective-C','ObjectiveC','objective-c']:
        return 'Objective-C'
    elif skill in ['JavaScript','javascript','ES6']:
        return 'JavaScript'
    elif skill in ['3Ds Max']:
        return '3Ds Max'
    elif skill in ['Sass(SCSS)', 'SCSS','Sass','SASS']:
        return 'SASS'
    elif skill in ['Rxswift', 'rxswift']:
        return 'Rxswift'
    elif skill in ['SQL 서버','Microsoft SQL Server']:
        return 'SQL 서버'
    elif skill in ['NoSQL', 'NoSql', 'nosql']:
        return 'NoSQL'
    elif skill in ['Sequelize.js','Sequelize']:
        return 'Sequelize.js'
    elif skill in ['Swift','SwiftUI', 'swift']:
        return 'Swift'
    elif skill in ['Tensorflow','Tensorflow Lite','TensorFlow', 'tensorflow']:
        return 'Tensorflow'
    elif skill in ['Django','django','DRF(Django REST framework)']:
        return 'Django'
    elif skill in ['jQuery', 'jquery ', 'jquery']:
        return 'jQuery'
    elif skill in ['Keras', 'keras']:
        return 'Keras'
    elif skill in ['Terraform']:
        return 'Terraform'
    elif skill in ['MariaDB','mariadb']:
        return 'MariaDB'
    elif skill in ['MATLAB','matlab']:
        return 'MATLAB'
    elif skill in ['MongoDB','mongodb']:
        return 'MongoDB'
    elif skill in ['MSSQL','mssql']:
        return 'MSSQL'
    elif skill in ['MySQL','mysql']:
        return 'MySQL'
    elif skill in ['Nginx','NGINX','nginx']:
        return 'Nginx'
    elif skill in ['Notion','notion.so']:
        return 'Notion'
    elif skill in ['OpenCV','opencv']:
        return 'OpenCV'
    elif skill in ['PHP','php']:
        return 'PHP'
    elif skill in ['PostgreSQL', 'postgresql']:
        return 'PostgreSQL'
    elif skill in ['R','r']:
        return 'R'
    elif skill in ['Redis','redis']:
        return 'Redis'
    elif skill in [ 'Unity','Unity3D','unity']:
        return 'Unity'
    elif skill in [ 'SQL', 'SQLite']:
        return 'SQL'
    elif skill in [ 'Go', 'Golang']:
        return 'Go'
    elif skill in ['CI/CD' ]:
        return 'CI/CD' 
    elif skill in [ 'Docker' ]:
        return  'Docker' 
    elif skill in ['ExpressJS']:
        return 'ExpressJS'
    elif skill in ['FastAPI' ]:
        return 'FastAPI' 
    elif skill in ['Firebase']:
        return 'Firebase'
    elif skill in ['Flask' ]:
        return 'Flask' 
    elif skill in ['JIRA' ,'Jira']:
        return 'JIRA'
    elif skill in ['Jenkins']:
        return 'Jenkins' 
    elif skill in ['Kubernetes' ]:
        return 'Kubernetes' 
    elif skill in ['Microservice Architecture' ]:
        return 'Microservice Architecture' 
    elif skill in ['Next.js' ]:
        return 'Next.js' 
    elif skill in ['OpenCL']:
        return 'OpenCL' 
    elif skill in ['OpenGL']:
        return 'OpenGL'
    elif skill in ['Redmine' ]:
        return 'Redmine' 
    elif skill in ['Rust' ]:
        return 'Rust' 
    elif skill in ['SVN' ]:
        return 'SVN' 
    elif skill in ['Selenium']:
        return 'Selenium'
    elif skill in ['TypeScript' ]:
        return 'TypeScript' 
    elif skill in ['3D','3D 모델링','API 개발','Big Data', 'Blockchain', 'Data Analysis','Data Analysys','Database','Deep Learning', 
                   'DevOps','DirectX','EEO','ERP 소프트웨어', 'ETL', 'Eclipse','Embedded System', 'Excel','GPU','GUI', 'Google Analytics',
                   'Google Apps', 'ISO','IT 관리','IT 운영','Image Processing','IntelliJ IDEA','IoT', 'JSON', 'ORM','OTN', 'PCB 디자인',
                   'PCB 레이아웃 설계','PKI(Public key infrastructure)','PLC','PMP','QA 엔지니어링', 'RDBMS','REST','REST API',
                   'RESTful Architecture', 'RESTful WebServices', 'RPC(Remote Procedure Call)', 'RTL 설계','RTL 코딩', 'React Admin',
                   'Restful API', 'Rx',  'SCM', 'SDN','SEO', 'SNMP','SONET','SPI', 'SSH','SSO', 'Scrum', 'Slack', 'Slack API', 'SoC', 
                   'Storage', 'TCP','TCP/IP','TDD',  'UDP','UI 디자인', 'Verilog', 'WPF','WPF 개발','Web Socket', 'WebSocket',
                   'WebSphere MQ', 'XML','XP', 'debugging','dlib',  'iOS','iOS 개발', '강의','개발','검증','고객 관계','고객 지원',
                   '공공 부문','공차 분석','교육 관리','교육 기술','구매 관리', '기술 개발','기술 관리', '기술 교육', '기술 문서',
                   '네비게이션 시스템', '네트워크 개발', '네트워크 관리','네트워크 보안', '네트워크 설계', '네트워크 운영',
                   '네트워크 인프라', '데이터베이스', '딥 러닝', '라이브러리 관리', '로봇', '로봇 프로그래밍', '리눅스 커널',
                   '마이크로프로세서','머신 비전', '모뎀', '모바일 기술', '모바일 장치', '백엔드 개발','보안', '보안 감사', '보안 관리',
                   '복제', '빅 데이터', '샘플 관리', '샘플 준비','생산 계획', '서버', '서버 관리', '서버 아키텍처', '서비스 관리',
                   '성능 측정', '소프트웨어 개발','솔루션 개발', '시스템 관리', '안드로이드 개발', '알고리즘 개발', '암호화', 
                   '연구 및 개발', '영상', '운영 관리', '운영체제', '웹 개발', '윈도우 모바일', '윈도우 프로그래밍', '유지보수', 
                   '인공 지능', '인프라','임베디드 소프트웨어', '임베디드 시스템', '자동차', '재고 관리', '전원 엔지니어링', 
                   '전자정부프레임워크', '정보 보안','정보 운영','정보관리','제안서 작성', '제어 시스템 설계', '제품 개발','차량',
                   '최적화', '취약점 스캐닝','카메라', '컴파일러', '컴퓨터 비전', '클라우드 보안', '테스트 실행', '통신', '펌웨어', 
                   '품질 관리', '프로그램 관리', '프로젝트 관리', '프론트엔드 개발자', '하드웨어', '회로', '회로 분석', '회로 설계',
                   'Agile','Children','Windows','Windows 8','Windows Embedded','Windows Server','Windows kernel', 'Windows 서버',
                   'BigData', 'DB', 'Etl','ai/인공지능','blockchain','ios','windows','3D Rendering','3D Volume Rendering','AI/인공지능',
                   'Cisco','DeepLearning','ERP','Embedded','HW','ISMS','MCU,''MQTT','MachineLearning','Matplotlib','Network','Optimize',
                   'Pads','PyCharm','QA','RDB','Red Hat Enterprise ','SW','Switch','switch','Smartcontract','VPN','VR','cisco','WebStorm',
                   'deeplearning','embedded','machinelearning', 'network', 'l2', 'l3','l4','rest api','sw','hw','WordPress', 'ADC', 'AJAX',
                   'API','AR', 'ARM', 'Adobe XD', 'Ajax', 'Appium', 'Aurora DB','Axios', 'Babel', 'Bash', 'BitBucket', 'Bootstrap', 'CAM',
                   'CRM','CUDA','CodeIgniter','Confluence','Consul','Coroutine', 'Cucumber', 'DACS', 'DART for Publishers', 
                   'DBA(Database administration)','DWDM', 'Dart','Delphi', 'Element UI', 'EnCase','Entity Framework', 'Ethereum','FFmpeg',
                   'FMEA','Flow','GStreamer','Hyperledger', 'I2C', 'IDA(Interactive DisAssembler)', 'IIS','IP', 'IPS','JSTL', 'JsonAPI', 
                   'Klaytn', 'Laravel', 'LiDAR', 'Lua', 'MES', 'MFC', 'ML', 'MVC', 'MVVM', 'MXNET', 'Machine Learning', 'Machine Vision', 
                   'Material-UI', 'Maven', 'Memcached', 'Moodle', 'MyBatis', 'NFC', 'NLP', 'NVIDIA TensorRT', 'Netty','OOP','OpenFlow',
                   'OpenMP', 'OpenStack','Oracle', 'Oracle Database', 'Perl', 'PowerBuilder', 'Prisma', 'Qt','RHEL', 'ROS','RS232', 'RTOS',
                   'RabbitMQ', 'STL', 'STP',  'SVG','Scala', 'Scikit-Learn', 'scikit-learn', 'Servlets', 'Shader', 'Simulink', 'Sketch', 'Socket.IO', 
                   'Solidity', 'Spa',  'Spinnaker', 'Storybook',  'Sybase',  'SLAM', 'TCL', 'Tableau','TeamCity', 'Tomcat', 'Truffle',
                   'TypeORM','UML', 'Ubuntu', 'Unreal Engine', 'VM웨어', 'Visual Basic', 'Visual Studio', 'Visual Studio Code','Vuetify.js',
                   'Web3.py','web3.js', 'WebGL', 'WebRTC', 'Webpack', 'WinForm', 'Xcode', 'Xilinx', 'Yarn', 'gRPC', 'DevExpress',
                   '고객 중심', 'Windows 서비스', 'EMC 규정 준수', '의료 장비', 'UX 디자인', '의료 영상', '비즈니스 분석', '설계', '보안 운영', '의료 기기'  ]:
        return None
    else:
        return skill

# 프로그래머스 데이터 확인

In [13]:
driver = webdriver.Chrome()
driver.get('https://career.programmers.co.kr/job?page=1&min_career=0&order=recent')
time.sleep(3)  # 3초 동안 대기

# 직무 버튼 누르기
buttons = driver.find_element(By.XPATH, '//*[@id="search-form"]/div[2]/div[1]/button')
buttons.click()

time.sleep(3)
html = driver.page_source

# 직무 딕셔너리 받아내기
soup = BeautifulSoup(html, 'lxml')
labels=soup.find_all('label', class_="job_category_label")
position_url = {}
for label in labels:
    position = classify_position(label.get_text(strip=True))
    if position is not None:
        url= "&job_category_ids=" + label.input['value']
        if position in position_url:
            position_url[position] = position_url[position] + url
        else:
            position_url[position] = url
            
print("position_url 딕셔너리를 완성했습니다!")

position_url 딕셔너리를 완성했습니다!


In [14]:
# 크롤링 시작
for position, url in position_url.items():
    print(f"{position} 공고 탐방을 시작하겠습니다!")
    URL=f'https://career.programmers.co.kr/job?page=1&min_career=0&order=recent{url}'
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    driver.get(URL)

    time.sleep(4)  # 3초 동안 대기
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 각 직무의 마지막 페이지 알아보기
    last_page=soup.find_all("span",class_="page-link")
    last_page=int(last_page[-2].get_text(strip=True))
    print(f"{position}의 마지막 페이지는 {last_page}!")
    
    url_num=[]
    
    # 각 직군의 전체 공고 url받아내기
    for page in range(1, last_page + 1):
        orig_url = f'https://career.programmers.co.kr/job?page={page}&min_career=0&order=recent{url}'
        driver.get(orig_url)
        imgcount = driver.find_elements(By.CSS_SELECTOR, '.list-positions')

        for i in imgcount:
            s = i.get_attribute('outerHTML')

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        divs = soup.find_all('ul', class_='list-positions')

        for div in divs:
            lists = div.find_all('li', class_='list-position-item')
            for list in lists:
                atag = list.find('a', class_='position-link')
                url_num.append(atag.attrs['href'].split('/')[-1])

        print(f"{position} 공고 {page}/{last_page}")
        
        n = 0
        
    print(f"{position}의 총 공고수는  {len(url_num)}개!")
    
    for url in url_num:
        URL = f'https://programmers.co.kr/job_positions/{url}'

        if URL in link_set:
            # 이미 존재하는 URL일 경우
            present_set.add(URL)
            print(f"아직 있는 공고입니다. URL: {URL}")
        else:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
            driver.get(url=URL)

            time.sleep(4)  # 3초 동안 대기

            driver.implicitly_wait(10)
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')

            # 공고명
            title_element = soup.select_one("h2")
            title = title_element.text if title_element else None

            # 회사명
            company = soup.select_one('h4.e_ow99Z6WyqEsY3oG3gk a').text

            # 지원 마감
            deadline = soup.find("div", text="지원 마감")
            deadline = deadline.find_next_sibling("div").text if deadline and deadline.find_next_sibling("div") else None

            # 근무 위치 정보 추출
            work_location = soup.find("div", text="근무 위치")
            work_location = work_location.find_next_sibling("div").text if work_location and work_location.find_next_sibling("div") else None

            # 기술 스택 정보 추출
            stacks = soup.find("ul", class_="cV112CI8tyAk3_QcrdKs")
            stack = []

            if stacks:
                for li_tag in stacks.find_all('li'):
                    skill = li_tag.text
                    classified_skill = classify_skill(skill)

                    if classified_skill is None:
                        pass
                    else:
                        if classified_skill not in stack:
                            stack.append(classified_skill)
            else:
                stack = None

            # 추출한 정보를 데이터프레임에 추가
            new_df.loc[len(new_df)] = [title, company, position, deadline, work_location, stack, URL]
            print(f"현재까지 모인 {position} 공고:{n+1}개!")

        n += 1

    print(f"{position} 공고 전체 끝!")

driver.quit()

백엔드 공고 탐방을 시작하겠습니다!
백엔드의 마지막 페이지는 5!
백엔드 공고 1/5
백엔드 공고 2/5
백엔드 공고 3/5
백엔드 공고 4/5
백엔드 공고 5/5
백엔드의 총 공고수는  96개!
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18459
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18147
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/15840
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/15845
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/1207
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/17780
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/17779
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/4351
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/12991
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/7771
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/11917
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/3669
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/16409
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/1608

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:70: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  deadline = soup.find("div", text="지원 마감")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:74: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  work_location = soup.find("div", text="근무 위치")


현재까지 모인 모바일 공고:7개!


C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:70: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  deadline = soup.find("div", text="지원 마감")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:74: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  work_location = soup.find("div", text="근무 위치")


현재까지 모인 모바일 공고:8개!
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/9637
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/16408
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/16407
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18765
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/15891
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/17290
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18171
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18764
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/9649
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18590
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18150
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/8930
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/12920
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/16655
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/3872
아직 있는 공고입니다. URL: https:/

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:70: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  deadline = soup.find("div", text="지원 마감")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:74: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  work_location = soup.find("div", text="근무 위치")


현재까지 모인 데이터 엔지니어 공고:6개!


C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:70: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  deadline = soup.find("div", text="지원 마감")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18728\3379677868.py:74: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  work_location = soup.find("div", text="근무 위치")


현재까지 모인 데이터 엔지니어 공고:7개!
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/12417
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18624
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/9647
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/18612
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/12783
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/17763
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/4023
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/17095
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/17097
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/16757
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/10133
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/15444
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/4871
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/14660
아직 있는 공고입니다. URL: https://programmers.co.kr/job_positions/14657
아직 있는 공고입니다. URL: h

# 원티드

In [15]:
driver = webdriver.Chrome()

# 페이지 열기
driver.get('https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=company.response_rate_order&years=0&locations=all')
time.sleep(3)  # 3초 동안 대기
    
# 직무 버튼 누르기
buttons = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/button/span[2]')
buttons.click()

time.sleep(3)
html = driver.page_source

# 직무 리스트 받아내기
soup = BeautifulSoup(html, 'lxml')
positions=soup.find_all('button', class_="JobCategoryItem_JobCategoryItem__oUaZr")
positions = [position.text for position in positions][1:]

In [16]:
done_position=set()

# 지역 저장 시 사용하는 keywords
keywords = ['대한민국', '서울', '경기', '제주', '울산', '경상북도', '경북', '부산', '인천', '대전', '경기']

for i in range(0,len(positions)):
    position = classify_position(positions[i])
    print(f"지금 포지션은 {position}")
    if position is not None:
        if i==0:
            pass
        else:
            driver.close()
            driver = webdriver.Chrome()

            # 페이지 열기
            driver.get('https://www.wanted.co.kr/wdlist/518/873?country=kr&job_sort=company.response_rate_order&years=0&locations=all')

            buttons = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/button')
            buttons.click()

            driver.implicitly_wait(10)

            button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[2]')
            button.click()

            button = driver.find_element(By.XPATH, f'//*[@id="__next"]/div[3]/article/div/div[2]/section/div[1]/div/button[{i+2}]')
            button.click()

            button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[2]/button/span[2]')
            button.click()
        print(f'{i+1}. 지금부터 {position} 시작!')

        # position 별 크롤링 하는 것

        # 스크롤 하는 횟수 세는 변수
        scroll_count = 0

        # 스크롤 내리기
        while True:
            # 스크롤 위치 저장
            last_height = driver.execute_script("return document.documentElement.scrollHeight")

            # 스크롤 내리기
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

            # 로딩 대기
            time.sleep(3)

            # 스크롤 위치 갱신
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            scroll_count += 1

            # 스크롤을 더 이상 내릴 수 없는 경우 종료
            if new_height == last_height or scroll_count == 10000:
                break


        # 페이지 내용 가져오기
        page_source = driver.page_source

        # BeautifulSoup을 사용하여 데이터 추출
        soup = BeautifulSoup(page_source, 'lxml')
        # 원하는 정보를 찾기 위해 적절한 BeautifulSoup 메소드를 사용합니다.

        a=soup.find("div", class_="List_List_container__JnQMS")
        href_selector = 'div.Card_className__u5rsb a[href]'
        href_values = [a['href'] for a in soup.select(href_selector)]

        # 결과 출력
        url_num=[href for href in href_values]
        print(f'{position} url주소 추출 끝!')

        print("이제 데이터 프레임 채워넣기를 시작하겠습니다.")

        # 데이터프레임 채워넣기 시작
        n=1
        for url in url_num:
            URL = f'https://www.wanted.co.kr//{url}'
            
            if URL in link_set:
                # 이미 존재하는 URL일 경우
                present_set.add(URL)
                print(f"아직 있는 공고입니다. URL: {URL}")
            else:
                
                if URL in new_df['링크'].values:
                    if position not in done_position:
                    # 이미 존재하는 URL인 경우 해당 position 값에 추가
                        existing_positions = new_df.loc[new_df['링크'] == URL, '직무']
                        updated_position = existing_positions + ", " + position
                        new_df.loc[new_df['링크'] == URL, '직무'] = updated_position
                        print("기존에 있는 공고이기에 position만 추가했습니다.")
                else:
                    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

                    # 웹 페이지 가져오기
                    response = requests.get(URL)
                    html_content = response.content

                    # BeautifulSoup 객체 생성
                    soup = BeautifulSoup(html_content, 'lxml')
                    script_content = soup.select("script", type='application/ld+json')[0]
                    data = json.loads(script_content.string)

                    # 공고명
                    title=data['title']
                    #회사명
                    company=data['hiringOrganization']['name']
                    #마감일
                    deadline=data['validThrough']
                    #근무지
                    address=data['jobLocation']['address']['streetAddress']

                    if address == '':
                        work_location = None

                    else: 
                        first_word = address.split()[0]
                        if any(keyword in first_word for keyword in keywords):
                            work_location = address
                        else:
                            region = data['jobLocation']['address']['addressRegion']
                            work_location = f"{region[:2]} {address}"

                    driver = webdriver.Chrome()
                    driver.get(url=URL)

                    html = driver.page_source
                    soup = BeautifulSoup(html, 'lxml')

                    # 기술 스택
                    stack_elements = soup.find_all("div", class_="SkillItem_SkillItem__E2WtM")
                    stack = []

                    for element in stack_elements:
                        skill = element.text
                        classified_skill = classify_skill(skill)

                        if classified_skill is None:
                            pass
                        else:
                            if classified_skill not in stack:
                                stack.append(classified_skill)


                    #추출한 정보를 데이터프레임에 추가
                    new_df.loc[len(new_df)] = [title, company, position, deadline, work_location, stack, URL]
                    print(f"{position}의 {n}번째 공고 정보 끝!")
                    n+=1

        print(f"{position} 데이터 프레임 생성 끝!")
        done_position.add(position)
    
driver.quit()

지금 포지션은 웹개발
1. 지금부터 웹개발 시작!
웹개발 url주소 추출 끝!
이제 데이터 프레임 채워넣기를 시작하겠습니다.
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/162828
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/162396
웹개발의 1번째 공고 정보 끝!
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/161517
웹개발의 2번째 공고 정보 끝!
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/159542
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/158859
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/156974
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/155695
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/155689
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/154875
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/154335
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/153995
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/147084
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/147083
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/145435
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/144049
아직 있는 공고입니다. URL: https://www.wanted.co.kr///wd/142205
아직 있는 공고입니다. U

# 점핏

In [17]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
orig_url = 'https://www.jumpit.co.kr/positions' # 크롤링 할 사이트

driver.get(orig_url)
time.sleep(2)

#스크롤 전 높이
before_h = driver.execute_script("return window.scrollY")  # 스크롤을 하지 않았으니 0이 변수에 대입

career = driver.find_element(By.XPATH, '/html/body/div[1]/main/div/div[1]/div/div[1]/div/div[1]/div[2]/button')
career.click()
time.sleep(2)

rookie = driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div/div/div[1]/div/div[1]/div[2]/div[1]/div[2]/div/div/div[2]/label')
rookie.click()
time.sleep(2)

first_click = driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div/div/div[1]/div/div[1]/div[2]/button')
first_click.click()
time.sleep(1)

add_list = []
pos = ['백엔드', '프론트엔드', '웹개발', '모바일', '모바일', '백엔드', '데이터 엔지니어', 'AI/ML', 'DevOps',]
pos_num = [2, 3, 4, 5, 6, 10, 11, 12, 13]
pos_dict = dict(zip(pos_num, pos))  # 딕셔너리 생성

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 18.1MB/s]


In [18]:
for i in pos_num:
    position = driver.find_element(By.XPATH, f'//*[@id="root"]/main/div/section/section/div[1]/button[{i}]')
    position_text = pos_dict[i]
    position.click()

    # 무한 스크롤
    prev_page_source = ""
    while True:
        # 맨 아래로 스크롤
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(2)

        # 페이지 소스 가져오기
        current_page_source = driver.page_source

        if current_page_source == prev_page_source:
            # 이전 페이지 소스와 현재 페이지 소스가 같으면 스크롤이 끝까지 이동한 것으로 판단하여 반복문을 탈출
            break

        prev_page_source = current_page_source

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('div.sc-fIosxK.fKzIXW')

    for item in items:
        add_elem = item.select('a')
        for add in add_elem:
            add_list.append(('https://www.jumpit.co.kr' + add['href'], position_text))

    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.HOME)
    time.sleep(1)
    position.click()
    time.sleep(2)

In [19]:
for orig_url, position in add_list:
    if orig_url in link_set:
        # 이미 존재하는 URL일 경우
        present_set.add(orig_url)
        print(f"아직 있는 공고입니다. URL: {orig_url}")
    else:
        driver.get(orig_url)
        time.sleep(4)
        title = driver.find_element(By.CSS_SELECTOR, 'section.sc-gnnDb > h1').text
        company = driver.find_element(By.CSS_SELECTOR, 'div.position_title_box_desc > a').text

        try:
            deadline_text = driver.find_element(By.CSS_SELECTOR, 'dl.sc-itWPBs:nth-child(4) > dd').text
            if re.match(r"\d{4}-\d{2}-\d{2}", deadline_text):
                deadline = deadline_text  # 날짜 형식이라면 해당 값을 그대로 사용
            else:
                deadline = "상시 채용"
        except NoSuchElementException:
                deadline = "상시 채용"

        try:    
            element = driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[2]/div/section[3]/dl[4]/dd/ul/li')
            work_location = element.text.replace('지도보기', '').replace('주소복사', '').replace('\.', '')
        except NoSuchElementException:
            work_location = None
        try:
                skill_elements = driver.find_elements(By.XPATH, '/html/body/div[1]/main/div/div[2]/div/section[2]/dl[1]/dd/pre/div')
                stack = []

                for element in skill_elements:
                    skill = element.text
                    classified_skill = classify_skill(skill)

                    if classified_skill is None:
                        pass
                    else:
                        if classified_skill not in stack:
                            stack.append(classified_skill)

            
        except NoSuchElementException:
            stack = []
        link = orig_url
    
        new_df.loc[len(new_df)] = [title, company, position, deadline, work_location, stack, URL]
        print(f"{position}의 {n}번째 공고 정보 끝!")
        
driver.quit()    

아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/595
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/16516
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/16273
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/16274
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/1622
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/9050
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/9581
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/10504
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/16901
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/16518
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/13886
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/16515
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/15293
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/14644
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/14546
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/11999
아직 있는 공고입니다. URL: https://www.jumpit.co.kr/position/16841
아직 있는 공고입니다. URL: h

# 삭제해야 하는 데이터프레임 json 저장

In [20]:
complement = link_set - present_set
delete_df = now_df[now_df['링크'].isin(complement)]
# 배경화면에 저장
delete_df.to_json(r"C:\Users\Playdata\Desktop\delete_df.json", orient="records")

# 새롭게 들어가는 데이터프레임

## 관련 함수

In [21]:
# 두 공고에 모두 있는 회사명 데이터 빼내기 filtered_df
def preprocess_and_compare_links(df,intersection):
    # 공통으로 있는 회사만 필터링
    filtered_df = df[df['회사명'].isin(intersection)]
    # NaN인 기술스택을 '[]'로 대체
    filtered_df.loc[filtered_df['기술스택'].apply(lambda x: isinstance(x, float)), '기술스택'] = filtered_df.loc[filtered_df['기술스택'].apply(lambda x: isinstance(x, float)), '기술스택'].apply(lambda x: '[]' if pd.isnull(x) else x)
    # 공고명에서 '신입'과 ()로 둘러싸인 부분 삭제
    filtered_df['공고명'] = filtered_df['공고명'].apply(lambda x: re.sub(r' \(신입.*?\)|신입|\[신입.*?\] |\[코스닥상장사\] |채용', '', x))
    filtered_df['공고명'] = filtered_df['공고명'].apply(lambda x: re.sub(r'\(신입/경력\)', '', x))

    # 중복되지 않은 데이터를 추출
    df = df[~df['링크'].isin(filtered_df['링크'])]
    
    return df,filtered_df

In [23]:
# 회사가 같고, 링크는 다르고, 한 공고문이 다른 공고문을 완전히 포함하고 있을 때 
# 중복된 공고명을 가진 그룹 찾기
def find_exact_matches(grouped, url_a, url_b):
    exact_match_df = pd.DataFrame(columns=grouped.first().columns)
    for _, group in grouped:
        if len(group) > 1:
            for i in range(len(group)):
                a_group = group.iloc[i]
                for j in range(i + 1, len(group)):
                    b_group = group.iloc[j]
                    if ((a_group['링크'].startswith(url_a) and b_group['링크'].startswith(url_b))
                        or (a_group['링크'].startswith(url_b) and b_group['링크'].startswith(url_a))):
                        if a_group['공고명'].replace(" ", "") == b_group['공고명'].replace(" ", ""):
                            if a_group['기술스택'] != b_group['기술스택']:
                                combined_tech_stack = set()

                                if isinstance(a_group['기술스택'], str):
                                    a_group['기술스택'] = ast.literal_eval(a_group['기술스택'])
                                    for a_stack in a_group['기술스택']:
                                        combined_tech_stack.add(a_stack)

                                if isinstance(b_group['기술스택'], str):
                                    b_group['기술스택'] = ast.literal_eval(b_group['기술스택'])
                                    for b_stack in b_group['기술스택']:
                                        combined_tech_stack.add(b_stack)
                                    
                                combined_tech_stack = list(combined_tech_stack) 

                                a_group['기술스택'] = combined_tech_stack
                                b_group['기술스택'] = combined_tech_stack

                            exact_match_df = exact_match_df.append(a_group)
                            exact_match_df = exact_match_df.append(b_group)

    return exact_match_df

In [24]:
# 포함관계인 공고문 찾기
def find_included_matches(grouped, url_a, url_b):
    included_df = pd.DataFrame(columns=grouped.first().columns)
    for _, group in grouped:
        if len(group) > 1:
            for i in range(len(group)):
                a_group = group.iloc[i]
                for j in range(i + 1, len(group)):
                    b_group = group.iloc[j]
                    if ((a_group['링크'].startswith(url_a) and b_group['링크'].startswith(url_b))
                        or (a_group['링크'].startswith(url_b) and b_group['링크'].startswith(url_a))):
                        if (a_group['공고명'].replace(" ", "") in b_group['공고명'].replace(" ", "") 
                        or b_group['공고명'].replace(" ", "") in a_group['공고명'].replace(" ", "")):
                            if a_group['기술스택'] != b_group['기술스택']:
                                combined_tech_stack = set()

                                if isinstance(a_group['기술스택'], str):
                                    a_group['기술스택'] = ast.literal_eval(a_group['기술스택'])
                                    for a_stack in a_group['기술스택']:
                                        combined_tech_stack.add(a_stack)

                                if isinstance(b_group['기술스택'], str):
                                    b_group['기술스택'] = ast.literal_eval(b_group['기술스택'])
                                    for b_stack in b_group['기술스택']:
                                        combined_tech_stack.add(b_stack)

                                combined_tech_stack = list(combined_tech_stack) 

                                a_group['기술스택'] = combined_tech_stack
                                b_group['기술스택'] = combined_tech_stack

                            included_df = included_df.append(a_group)
                            included_df = included_df.append(b_group)
    return included_df

In [25]:
def process_df(filtered_df, exact_match_df, included_df, link_start):
    solo_df = filtered_df[~filtered_df['링크'].isin(exact_match_df['링크'])]
    solo_df = solo_df[~solo_df['링크'].isin(included_df['링크'])]

    exact_match_df['기술스택'] = exact_match_df['기술스택'].apply(lambda x: str(x))
    included_df['기술스택'] = included_df['기술스택'].apply(lambda x: str(x))

    duplicates_df = pd.merge(exact_match_df, included_df, how='outer')

    duplicates_df = duplicates_df[duplicates_df['링크'].str.startswith(link_start)]

    # solo_df를 더함
    final_df = df.append(solo_df, ignore_index=True)  # 이 부분을 수정했습니다.

    # duplicates_df를 더함
    final_df = final_df.append(duplicates_df, ignore_index=True)
    
    return final_df

## 새로운 데이터끼리 같은 공고 구분

In [26]:
df1 = new_df[new_df['링크'].str.startswith('https://programmers.co.kr/')]
df2 = new_df[new_df['링크'].str.startswith('https://www.wanted.co.kr/')]
df3 = new_df[new_df['링크'].str.startswith('https://jumpit.co.kr')]

In [27]:
if (len(df1) !=0) and (len(df2) !=0):
    intersection = pd.Series(list(set(df1['회사명'].unique()) & set(df2['회사명'].unique())))
    if len(intersection) !=0:
        new_df, filtered_df = preprocess_and_compare_links(new_df, intersection)
        grouped = filtered_df.groupby('회사명')
        
        exact_match_df = find_exact_matches(grouped, 'https://programmers.co.kr', 'https://www.wanted.co.kr')
        
        included_df = find_included_matches(grouped, 'https://programmers.co.kr', 'https://www.wanted.co.kr')
        
        new_df = process_df(filtered_df, exact_match_df, included_df, 'https://programmers.co.kr')

In [28]:
if (len(df1) != 0) and (len(df3) != 0):
    new_df['근무지'] = new_df['근무지'].str.replace('\n·', '')
    intersection = pd.Series(list(set(df['회사명'].unique()) & set(df3['회사명'].unique())))
    if len(intersection) !=0:
        new_df,filtered_df=preprocess_and_compare_links(new_df,intersection)
        # 회사가 같고, 링크는 다르고, 한 공고문이 다른 공고문을 완전히 포함하고 있을 때 
        # 중복된 공고명을 가진 그룹 찾기
        grouped = filtered_df.groupby('회사명')

        # 완전히 일치하는 경우를 찾는 코드
        exact_match_df = pd.DataFrame(columns=filtered_df.columns)
        exact_match_df = exact_match_df.append(find_exact_matches(grouped, 'https://programmers.co.kr', 'https://www.jumpit.co.kr'))

        # 포함관계인 경우를 찾는 코드
        included_df = pd.DataFrame(columns=filtered_df.columns)
        included_df = included_df.append(find_included_matches(grouped, 'https://programmers.co.kr', 'https://www.jumpit.co.kr'))

        new_df = process_df(filtered_df, exact_match_df, included_df, 'https://www.jumpit.co.kr')


In [30]:
if (len(df2) != 0) and (len(df3) != 0):
    new_df['근무지'] = new_df['근무지'].str.replace('\n·', '')
    intersection = pd.Series(list(set(df['회사명'].unique()) & set(df3['회사명'].unique())))
    if len(intersection) !=0:
        new_df,filtered_df=preprocess_and_compare_links(new_df,intersection)
        # 회사가 같고, 링크는 다르고, 한 공고문이 다른 공고문을 완전히 포함하고 있을 때 
        # 중복된 공고명을 가진 그룹 찾기
        grouped = filtered_df.groupby('회사명')

        # 완전히 일치하는 경우를 찾는 코드
        exact_match_df = pd.DataFrame(columns=filtered_df.columns)
        exact_match_df = exact_match_df.append(find_exact_matches(grouped, 'https://www.wanted.co.kr', 'https://www.jumpit.co.kr'))

        # 포함관계인 경우를 찾는 코드
        included_df = pd.DataFrame(columns=filtered_df.columns)
        included_df = included_df.append(find_included_matches(grouped, 'https://www.wanted.co.kr', 'https://www.jumpit.co.kr'))
        
        new_df = process_df(filtered_df, exact_match_df, included_df, 'https://www.jumpit.co.kr')

## new_df 전처리

### 직무 전처리

In [31]:
new_df['직무'] = new_df['직무'].str.split(', ')
# 웹개발 혼자 있을 때
new_df['직무'] = new_df['직무'].apply(lambda x: ["프론트엔드", "백엔드"] if str(x) == str(["웹개발"]) else x)
# 웹개발이 다른 직무와 같이 있을 때
new_df['직무'] = new_df['직무'].apply(lambda x: [item for item in x if item != '웹개발'])

In [32]:
new_df.to_csv(r'C:\Users\Playdata\Desktop\df_pos.csv', index=False, encoding='cp949')
new_df=pd.read_csv(r'C:\Users\Playdata\Desktop\df_pos.csv', encoding='cp949')

new_df['직무'] = new_df['직무'].str.replace("'", '"')

### 기술스택 전처리

In [33]:
# 기술 스택 획일화
# np.where(condition, x, y)를 활용해서 condition이 참일 경우 x를, 아닌 경우 y로!
new_df['기술스택'] = new_df['기술스택'].str.replace("'", '"')
new_df['기술스택'] = np.where((new_df['기술스택'].isnull()) | (new_df['기술스택'] == "[]"), """[""]""", new_df['기술스택'])

### 마감일 전처리

In [34]:
new_df['마감일'] = new_df['마감일'].str.replace('상시', '상시 채용')
new_df['마감일'] = new_df['마감일'].str.replace('채용 채용', '채용')
new_df['마감일'] = new_df['마감일'].fillna('상시 채용')

# 날짜 형식 2023-04-12
def format_date(date_str):
    if date_str.startswith('상시 채용'):
        return date_str
#     elif date_str.startswith('상시'):
#         return date_str
    elif ':' in date_str:
        date_obj = datetime.datetime.strptime(date_str, '%y년 %m월 %d일 %H:%M까지')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    else:
        return date_str

new_df['마감일'] = new_df['마감일'].apply(format_date)

### 근무지 전처리

In [36]:
new_df['근무지'] = new_df['근무지'].str.replace('주소 ', '')
new_df['근무지'] = new_df['근무지'].str.replace('대한민국 ', '')
new_df['근무지'] = new_df['근무지'].str.replace('서울특별시', '서울')
new_df['근무지'] = new_df['근무지'].str.replace('서울시', '서울')
new_df['근무지'] = new_df['근무지'].str.replace('경기도', '경기')
new_df['근무지'] = new_df['근무지'].str.replace('제주특별자치도', '제주')
new_df['근무지'] = new_df['근무지'].str.replace('제주특별자치도', '제주')
new_df['근무지'] = new_df['근무지'].str.replace('울산광역시', '울산')
new_df['근무지'] = new_df['근무지'].str.replace('경상북도', '경북')
new_df['근무지'] = new_df['근무지'].str.replace('부산광역시', '부산')
new_df['근무지'] = new_df['근무지'].str.replace('인천광역시', '인천')
new_df['근무지'] = new_df['근무지'].str.replace('대전시', '대전')
new_df['근무지'] = new_df['근무지'].str.replace('대전광역시', '대전')
new_df['근무지'] = new_df['근무지'].str.replace('대구광역시', '대구')
new_df['근무지'] = new_df['근무지'].str.replace('서울 경기', '경기')
new_df['근무지'] = new_df['근무지'].str.replace('전라남도', '전남')
new_df['근무지'] = new_df['근무지'].str.replace('인천시', '인천')
new_df['근무지'] = new_df['근무지'].str.replace('Level ', '')
new_df['근무지'] = new_df['근무지'].str.replace('입니다', '')
new_df['근무지'] = new_df['근무지'].str.replace('입니다', '')
new_df['근무지'] = new_df['근무지'].str.replace('대구 대전', '대전')
new_df['근무지'] = new_df['근무지'].str.replace('서울 06237', '')
new_df['근무지'] = new_df['근무지'].str.replace(' 서울', '서울')
new_df['근무지'] = new_df['근무지'].str.replace(r'서울 \(06159\)\s*', '')
new_df['근무지'] = new_df['근무지'].str.replace(r'서울 13449\)\s*', '')
new_df['근무지'] = new_df['근무지'].str.replace(r'\(07807\)\s*', '')
new_df['근무지'] = new_df['근무지'].str.replace(r'\(08380\)\s*', '')
new_df['근무지'] = new_df['근무지'].str.replace(r'\(06159\)\s*', '')
new_df['근무지'] = new_df['근무지'].str.replace(r'\[본사\]\s*', '')
new_df['근무지'] = new_df['근무지'].str.replace('\n·', '')
new_df['근무지'] = new_df['근무지'].str.replace(r'^\(\d+\)\s*', '', regex=True)
new_df['근무지'] = new_df['근무지'].str.replace(r'\[.*?\]', '', regex=True)
new_df['근무지'] = new_df['근무지'].str.replace('연남로13길9', '서울 마포구 연남로13길9', regex=True)
new_df['근무지'] = new_df['근무지'].str.replace('광주 광주광역시', '광주')
new_df['근무지'] = new_df['근무지'].str.replace('서울 논현로', '서울 강남구 논현로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 왕십리로', '서울 성동구 왕십리로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 서초동', '서울 서초구 서초동')
new_df['근무지'] = new_df['근무지'].str.replace('서울 마포대로', '서울 마포구 마포대로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 반포대로', '서울 서초구 반포대로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 원격 근무', '원격 근무')
new_df['근무지'] = new_df['근무지'].str.replace('서울 개포로', '서울 강남구 개포로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 삼성로', '서울 강남구 삼성로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 양재동', '서울 서초구 양재동')
new_df['근무지'] = new_df['근무지'].str.replace('경기 금토로', '경기 성남시 수정구 금토로')
new_df['근무지'] = new_df['근무지'].str.replace('대전 가정북로', '대전 유성구 가정북로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 강남대로 156', '서울 서초구 강남대로 156')
new_df['근무지'] = new_df['근무지'].str.replace('경기 판교', '경기 성남시 분당구 판교동')
new_df['근무지'] = new_df['근무지'].str.replace('대구 대구시내', '대구 중구 동성로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 양화로', '서울 마포구 양화로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 발산역', '서울 강서구 마곡동')
new_df['근무지'] = new_df['근무지'].str.replace('서울 양재역', '서울 서초구 양재역')
new_df['근무지'] = new_df['근무지'].str.replace('서울 서울', '서울')
new_df['근무지'] = new_df['근무지'].str.replace('서울 선릉로', '서울 강남구 선릉로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 테헤란로', '서울 강남구 테헤란로')
new_df['근무지'] = new_df['근무지'].str.replace('서울 삼성동', '서울 강남구 삼성동')
new_df['근무지'] = new_df['근무지'].str.replace('서울 양재동', '서울 서초구 양재동')

new_df['근무지'] = new_df['근무지'].apply(lambda x: x.split(' ', 1)[1] if len(x.split(' ')) > 1 and x.split(' ')[0] == x.split(' ')[1] else x)

new_df.loc[new_df['근무지'].str.contains('Seoul, Republic of Korea', na=False), '근무지'] = None
new_df.loc[new_df['근무지'].str.contains('서울 대치로 223', na=False), '근무지'] = None
new_df.loc[new_df['근무지'].str.contains('원격 근무', na=False), '근무지'] = None
new_df.loc[new_df['근무지'].str.contains('onetkorea137@gmail.com', na=False), '근무지'] = None

new_df['근무지'] = new_df['근무지'].astype(str)

# 기존 데이터와 new_df 비교

## new_df에만 있는 회사

In [37]:
new_company_list = pd.Index(new_df['회사명'].unique()).difference(now_df['회사명'].unique()).tolist()
new_company_list

['모빌테크', '생활연구소(청소연구소)', '아이테크', '위메이드(WEMADE)', '지란지교데이터', '트럼피아']

In [38]:
# 회사명과 평점으로 구성된 데이터 프레임
company_star_df = pd.DataFrame(columns=['회사명', '평점'])

# 크롤링 시작
driver = webdriver.Chrome()

driver.get('https://www.jobplanet.co.kr/search?query=+&category=search_new&search_keyword_hint_id=&_rs_con=seach&_rs_act=keyword_search')
time.sleep(3)  # 3초 동안 대기

for company in new_company_list:
    # 검색 버튼 누르기
    input_button = driver.find_element(By.XPATH, '//*[@id="search_bar_search_query"]')
    input_button.send_keys(company)
    time.sleep(2)
    star=0.0

    elements = driver.find_elements(By.CSS_SELECTOR, 'ul > li.company')
    if len(elements)==0:
        pass
    else:
        submit_button=driver.find_element(By.CSS_SELECTOR, '#search_form > div > button')
        submit_button.click()
        time.sleep(2)  # 2초 동안 대기
        if len(elements)==1:
            star=driver.find_element(By.XPATH,'//*[@id="mainContents"]/div[1]/div/div[2]/div[1]/div/span[3]').text        
        else:
            cards = driver.find_elements(By.CLASS_NAME, 'result_card')
            for card in cards:
                jp_company = card.text.split()[0].replace("(주)", "")
                if company == jp_company:
                    star = card.text.split()[2]
                    break

    print(company,star)    
    company_star_df.loc[len(company_star_df)] = [company, star]
    # 검색창 지우기
    input_button = driver.find_element(By.XPATH, '//*[@id="search_bar_search_query"]')  # input_button 웹 요소를 다시 찾음
    input_button.clear()
driver.quit()

모빌테크 2.6
생활연구소(청소연구소) 0.0
아이테크 0.0
위메이드(WEMADE) 0.0
지란지교데이터 4.7
트럼피아 2.1


In [39]:
# new_df에 해당 데이터 추가
new_df = new_df.merge(company_star_df, on='회사명', how='left')

In [53]:
## 회사명이 기존 데이터와 신규 데이터에 모두 있는 경우
merged_df = pd.merge(now_df, new_df, on='회사명', how='inner')
intersection = merged_df['회사명'].unique()
intersection = pd.Series(intersection)

In [54]:
intersection

0     앤씨앤
1    위메이드
dtype: object

In [55]:
allowed_link_starts = ['https://www.wanted.co.kr/', 'https://programmers.co.kr', 'https://www.jumpit.co.kr/']

for company in intersection:
    now_group = now_df[now_df['회사명'] == company]
    new_group = new_df[new_df['회사명'] == company]
    
    for _, now_row in now_group.iterrows():
        now_job_title = now_row['공고명'].replace(" ", "")
        
        for _, new_row in new_group.iterrows():
            new_job_title = new_row['공고명'].replace(" ", "")
            
            if now_job_title == new_job_title or now_job_title in new_job_title or new_job_title in now_job_title:
                matching_links = []
                if now_row['링크'].startswith('http'):
                    matching_links.append(now_row['링크'])
                if new_row['링크'].startswith('http'):
                    matching_links.append(new_row['링크'])
                
                if len(matching_links) > 1:
                    same_start = True
                    for i in range(1, len(matching_links)):
                        if not matching_links[i].startswith(matching_links[0]):
                            same_start = False
                            break
                    
                    if same_start:
                        continue
                
                if any(link.startswith(start) for link in matching_links for start in allowed_link_starts):
                    continue
                
                new_df.drop(new_row.name, inplace=True)

In [59]:
# 평점 넣기
merged_df = pd.merge(new_df, now_df[['회사명', '평점']], on='회사명', how='left')
merged_df['평점'] = merged_df['평점_y'].fillna(merged_df['평점_x'])
updated_df = merged_df.drop(['평점_x', '평점_y'], axis=1)


# 지역 추가 및 new_df

In [44]:
new_df['지역']=new_df['근무지'].apply(lambda address: " ".join(address.split(" ")[:2]))
new_df.to_csv(r'C:\Users\Playdata\Desktop\new_df.csv', index=True, encoding='cp949')

In [60]:
new_df

,공고명,회사명,직무,마감일,근무지,기술스택,링크,평점,지역
0,iOS SDK 개발,위메이드,"[""모바일""]",2023-09-07,"경기 성남시 분당구 대왕판교로644번길 49, 2층 201호 전체(삼평동, 코리아벤...","[""""]",https://programmers.co.kr/job_positions/18827,2.5,경기 성남시
1,Android SDK 개발,위메이드,"[""모바일""]",2023-09-07,"경기 성남시 분당구 대왕판교로644번길 49, 2층 201호 전체(삼평동, 코리아벤...","[""Kotlin""]",https://programmers.co.kr/job_positions/18826,2.5,경기 성남시
2,센서 융합 연구원,모빌테크,"[""데이터 엔지니어""]",상시 채용,"서울 서초구 마방로4길 16-9, (양재동)","[""C / C++"", ""Linux""]",https://programmers.co.kr/job_positions/17428,2.6,서울 서초구
3,QT 개발자(자율주행 센서데이터),모빌테크,"[""데이터 엔지니어""]",상시 채용,"서울 서초구 마방로4길 16-9, (양재동)","[""C / C++"", ""Linux""]",https://programmers.co.kr/job_positions/18820,2.6,서울 서초구
4,기술지원 엔지니어 - 판교,지란지교데이터,"[""프론트엔드"", ""백엔드""]",2023-06-15,경기 성남시 수정구 금토로 80번길 37 인피니티타워 W동 10층,"[""Git"", ""JIRA"", ""Java"", ""JavaScript"", ""C / C++...",https://www.wanted.co.kr///wd/162165,4.7,경기 성남시
5,기술지원 엔지니어 신입 - 대전,지란지교데이터,"[""프론트엔드"", ""백엔드""]",2023-06-15,대전 유성구 테크노 3로 65 한신에스메카 604,"[""Git"", ""JIRA"", ""Java"", ""JavaScript"", ""C / C++...",https://www.wanted.co.kr///wd/160496,4.7,대전 유성구
6,서버개발자,생활연구소(청소연구소),"[""백엔드"", ""프론트엔드""]",2023-06-30,경기 성남시 분당구 판교로 242 판교디지털센터 C동 10층 (주)생활연구소,"[""Git"", ""Linux"", ""MySQL"", ""React"", ""Ruby on Ra...",https://www.wanted.co.kr///wd/163956,0.0,경기 성남시
7,QA(신입가능),트럼피아,"[""프론트엔드"", ""백엔드""]",상시 채용,서울 강남구 테헤란로 441 11층,"[""Git"", ""ExpressJS"", ""Linux"", ""MySQL"", ""Redis""...",https://www.wanted.co.kr///wd/164226,2.1,서울 강남구
8,QA,위메이드(WEMADE),"[""프론트엔드"", ""백엔드""]",2023-09-07,경기 성남시 분당구 대왕판교로 644번길 49 위메이드타워,"[""""]",https://www.wanted.co.kr///wd/164353,0.0,경기 성남시
9,React 개발자,아이테크,"[""프론트엔드""]",2023-06-24,"서울 금천구 가산디지털1로 225, 제8층 제801호, 제802호, 제803호","[""React Native"", ""React"", ""Kotlin"", ""Swift"", ""...",https://www.wanted.co.kr///wd/8138,0.0,서울 금천구
